# LAB06: Know Your Data

In [ ]:
%pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Know your data').getOrCreate()

## Download Dataset

In [ ]:
! mkdir -p dataset

In [ ]:
! wget -P dataset https://www.dropbox.com/s/084rsqludghubkg/us_youtube_1K.csv

In [ ]:
! head dataset/us_youtube_1K.csv

In [ ]:
! wc -l dataset/us_youtube_1K.csv

## Create youtube Table

In [ ]:
df_youtube = spark.read.csv(path="dataset/us_youtube_1K.csv", header = True, inferSchema = True)
df_youtube.createOrReplaceTempView("youtube")

### Show table description

In [ ]:
spark.sql("\
DESC youtube \
").show()

### View first 5 records

In [ ]:
spark.sql("\
SELECT * FROM youtube LIMIT 5 \
").show()

### Count number of records

In [ ]:
spark.sql("\
SELECT count(1) num_rec FROM youtube \
").show()

## Know Your Data

### Get video_id length

In [ ]:
spark.sql("\
SELECT LENGTH(video_id) as len_rec, count(*) as num_rec \
FROM youtube \
GROUP BY LENGTH(video_id) \
ORDER BY LENGTH(video_id) \
").show(10)


### Show records where the video_id length equal to 13

In [ ]:
spark.sql("\
SELECT * \
FROM youtube \
WHERE LENGTH(video_id) = 13 \
LIMIT 5 \
").show()


### Show records where the video_id length is null

In [ ]:
spark.sql("\
SELECT * \
FROM youtube \
WHERE LENGTH(video_id) is null \
LIMIT 5 \
").show()


### Show records where the video_id length equal to 11

In [ ]:
spark.sql("\
SELECT * \
FROM youtube \
WHERE LENGTH(video_id) = 11 \
LIMIT 5 \
").show()


### Count number of record where the video_id length equal to 11

In [ ]:
spark.sql("\
SELECT count(*) valid_rec \
FROM youtube \
WHERE LENGTH(video_id) = 11 \
").show()


### View some column length where video_id length is 11

In [ ]:
spark.sql("\
SELECT LENGTH(video_id) len_video_id, LENGTH(trending_date) len_trending_date, LENGTH(category_id) len_category_id, LENGTH(publish_time) len_publish_time, LENGTH(views) len_views, \
       LENGTH(likes) len_likes, LENGTH(dislikes) len_dislike, count(*) num_rec \
FROM youtube \
GROUP BY LENGTH(video_id), LENGTH(trending_date), LENGTH(category_id), LENGTH(publish_time), LENGTH(views), LENGTH(likes), LENGTH(dislikes) \
HAVING len_video_id = 11 \
").show()


### Show records where video_id length is 11 and views length is 6

In [ ]:
spark.sql("\
SELECT * \
FROM youtube \
WHERE LENGTH(video_id) = 11 AND LENGTH(views) = 6 \
LIMIT 5 \
").show()

## Create New Table for Valid Data

### Query valid records

In [ ]:
spark.sql("\
SELECT \
  video_id, \
  CAST(concat( substr(trending_date,1,2), \
          substr(trending_date,7,2), \
          substr(trending_date,4,2)) \
          as int) trending_date, \
  title, \
  channel_title, \
  CAST(category_id as int), \
  publish_time, \
  tags, \
  CAST(views as int), \
  CAST(likes as int), \
  CAST(dislikes as int), \
  CAST(comment_count as int) \
FROM youtube \
WHERE LENGTH(video_id) = 11 AND LENGTH(trending_date) = 8 \
").show(5)

### Create stg_youtube table

In [ ]:
df_stg_youtube = spark.sql("\
SELECT \
  video_id, \
  CAST(concat( substr(trending_date,1,2), \
          substr(trending_date,7,2), \
          substr(trending_date,4,2)) \
          as int) trending_date, \
  title, \
  channel_title, \
  CAST(category_id as int), \
  publish_time, \
  tags, \
  CAST(views as int), \
  CAST(likes as int), \
  CAST(dislikes as int), \
  CAST(comment_count as int) \
FROM youtube \
WHERE LENGTH(video_id) = 11 AND LENGTH(trending_date) = 8 \
")

In [ ]:
df_stg_youtube.createOrReplaceTempView("stg_youtube")

### View stg_youtube table description

In [ ]:
spark.sql("\
DESC stg_youtube \
").show()

### Count number of records

In [ ]:
spark.sql("\
SELECT count(*) num_rec FROM stg_youtube \
").show()


### How many days in dataset?

In [ ]:
spark.sql("\
select count(1) num_days \
from( \
 select trending_date from stg_youtube \
 group by trending_date \
) a \
").show()


### Get number of records for each day

In [ ]:
spark.sql("\
select \
    trending_date, \
    count(*) num_rec \
from stg_youtube \
group by trending_date \
order by trending_date \
").show()